In [1]:
import tensorflow as tf

In [2]:
from songs import *

import copy
def addAllTransposedVersions(Songs, song):
    while song.transpose(1):
        pass

    Songs.append(copy.deepcopy(song))
    while song.transpose(-1):
        Songs.append(copy.deepcopy(song))
        
Songs = []
for i in range(1, 35):
    addAllTransposedVersions(Songs, Song('basic midi/track (' + str(i) + ').mid'))
    
all_songs = np.array([song.notes for song in Songs])

ERROR! out of range!
ERROR! out of range!
ERROR! out of range!


In [3]:
def generate():
    X = np.random.randint(0, 2, (128, 128+64, 13))
    X[:, 128:, :] = 0
    Y = np.zeros((128, 128+64, 13))
    Y[:, 64:, :] = X[:, :128, :]
    return X, Y

In [8]:
tf.reset_default_graph()

# Training Parameters
learning_rate = 0.001
training_steps = 10000
batch_size = 128
display_step = 200

# Network Parameters
num_input = 13
timesteps = 128+64 # timesteps
num_hidden = 50 # hidden layer num of features
history_size = 80

# tf Graph input
X = tf.placeholder("float", [batch_size, timesteps, num_input])
Y = tf.placeholder("float", [batch_size, timesteps, num_input])

loss_op = tf.constant(0.0)

read_lstm = tf.contrib.rnn.BasicLSTMCell(num_hidden, forget_bias=1.0, reuse=None)
read_lstm_state = tf.zeros([batch_size, num_hidden]), tf.zeros([batch_size, num_hidden])

read_w = tf.Variable(tf.random_normal([num_hidden, history_size]))
read_b = tf.Variable(tf.random_normal([history_size]))

history = tf.zeros([batch_size, num_input, history_size])

delete_lstm = tf.contrib.rnn.BasicLSTMCell(num_hidden, forget_bias=1.0, reuse=True)
delete_lstm_state = tf.zeros([batch_size, num_hidden]), tf.zeros([batch_size, num_hidden])

delete_w = tf.Variable(tf.random_normal([num_hidden, history_size]))
delete_b = tf.Variable(tf.random_normal([history_size]))

write_lstm = tf.contrib.rnn.BasicLSTMCell(num_hidden, forget_bias=1.0, reuse=True)
write_lstm_state = tf.zeros([batch_size, num_hidden]), tf.zeros([batch_size, num_hidden])

write_w = tf.Variable(tf.random_normal([num_hidden, history_size]))
write_b = tf.Variable(tf.random_normal([history_size]))

_X = tf.unstack(X, timesteps, 1)
_Y = tf.unstack(Y, timesteps, 1)
for t, inp, truth in zip(np.arange(timesteps), _X, _Y):
    read_lstm_output, read_lstm_state = read_lstm(inp, read_lstm_state)
    
    read = tf.sigmoid(tf.matmul(read_lstm_output, read_w) + read_b)
    read_proba = tf.nn.softmax(read, 1)
    loss_op -= tf.reduce_mean(tf.log(read_proba) * read_proba)
    memory_retrieve = tf.matmul(history, tf.expand_dims(read, 2))
    if t >= 64:
        loss_op += tf.reduce_mean(tf.squared_difference(tf.squeeze(memory_retrieve, 2), truth))
        
    delete_lstm_output, delete_lstm_state = delete_lstm(inp, delete_lstm_state)
    
    delete = tf.sigmoid(tf.matmul(delete_lstm_output, delete_w) + delete_b)
    delete_proba = tf.nn.softmax(delete, 1)
    loss_op -= tf.reduce_mean(tf.log(delete_proba) * delete_proba)
    history = history * tf.expand_dims(delete, 1)
    
    write_lstm_output, write_lstm_state = write_lstm(inp, write_lstm_state)
    write = tf.sigmoid(tf.matmul(write_lstm_output, write_w) + write_b)
    write_proba = tf.nn.softmax(write, 1)
    loss_op -= tf.reduce_mean(tf.log(write_proba) * write_proba)
    history = history + tf.tile(tf.expand_dims(inp, 2), [1, 1, history_size]) * tf.expand_dims(write, 1)

# Define loss and optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = tf.contrib.layers.optimize_loss(
    loss_op, tf.contrib.framework.get_global_step(), optimizer=optimizer,
    learning_rate=learning_rate)

init = tf.global_variables_initializer()

In [9]:
# Start training
sess = tf.Session()
# Run the initializer
sess.run(init)

for step in range(0, training_steps):
    batch_x, batch_y = generate()
    batch_x = batch_x.reshape((batch_size, timesteps, num_input))
    batch_y = batch_y.reshape((batch_size, timesteps, num_input))
    
    # Run optimization op (backprop)
    sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
    if step % display_step == 0:
        # Calculate batch loss and accuracy
        L = sess.run(loss_op, feed_dict={X: batch_x, Y: batch_y})
        print("Step " + str(step) + ", Minibatch Loss= " + "{:.4f}".format(L))

print("Optimization Finished!")

Step 0, Minibatch Loss= 89.6539
Step 200, Minibatch Loss= 87.4312
Step 400, Minibatch Loss= 87.1355
Step 600, Minibatch Loss= 86.8927
Step 800, Minibatch Loss= 86.1247
Step 1000, Minibatch Loss= 87.0389
Step 1200, Minibatch Loss= 86.9196
Step 1400, Minibatch Loss= 86.8620


KeyboardInterrupt: 

In [10]:
mr = sess.run(memory_retrieve, feed_dict={X: batch_x, Y: batch_y})